In [ ]:
GROQ_API_KEY = "YOUR_GROQ_API_KEY"

In [1]:
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew

def detect_purchase_orders(text):
    try:
        llm = ChatGroq(
            api_key=GROQ_API_KEY,
            model="claude-3-5-sonnet-20240620"
        )
        po_detector_agent = Agent(
            role='Purchase Order Identification Specialist',
            goal='Identify and categorize potential purchase order numbers in a given text',
            backstory='An expert in analyzing text to extract and validate potential purchase order identifiers',
            verbose=True
        )

        detect_task = Task(
            description=f'''
            Carefully analyze the following text and provide a comprehensive assessment:
            
            1. Identify any potential purchase order numbers or identifiers
            2. Determine the number of unique purchase order identifiers
            3. Provide reasoning for your identification
            4. Be precise and thorough in your analysis
            
            Input Text: {text}
            ''',
            agent=po_detector_agent,
            expected_output='A detailed breakdown of purchase order number detection'
        )

        crew = Crew(
            agents=[po_detector_agent],
            tasks=[detect_task]
        )

        analysis = crew.kickoff()

        def process_detection_result():
            if not analysis or 'no purchase order' in analysis.lower():
                return {
                    'status': 'NO_PO',
                    'purchase_order_numbers': None,
                    'count': 0
                }
            
            potential_pos = [
                item.strip() for item in analysis.split() 
                if item.strip()  
            ]
            
            unique_pos = list(dict.fromkeys(potential_pos))
            
            if len(unique_pos) == 1:
                return {
                    'status': 'SINGLE_PO',
                    'purchase_order_numbers': unique_pos,
                    'count': 1
                }
            elif len(unique_pos) > 1:
                return {
                    'status': 'MULTIPLE_POS',
                    'purchase_order_numbers': unique_pos,
                    'count': len(unique_pos)
                }
            
            return {
                'status': 'NO_PO',
                'purchase_order_numbers': None,
                'count': 0
            }

        return process_detection_result()

    except Exception as e:
        print(f"Error in purchase order detection: {e}")
        return {
            'status': 'ERROR',
            'purchase_order_numbers': None,
            'count': 0,
            'error_message': str(e)
        }

ModuleNotFoundError: No module named 'langchain_groq'

In [ ]:
input_text = "Order 4000114151 needs processing"
result = detect_purchase_orders(input_text)
print("Input:", input_text)
print("Detection Result:", result)